# Topic Modeling Analysis Notebook
# <font color='red'> *NOTE: Make sure to update the trt_API files!* </font>

#### This notebook contains a demonstration of the tools necessary for conducting clustering on Twitter data. 

In [ ]:
import trt_API.process as proc
import trt_API.analysis as ana
import trt_API.clustering as cluster
import trt_API.org_research as org 

import datetime
from sklearn.feature_extraction import text 

SORTED = False

## PREPROCESSING

### Variables for Analysis

In [ ]:
# Set the path to the parent directory containing all Tweets of interest
DIRECTORY = './tweets/*'
# Set to True to isolate english language tweets
ENGLISH = True

### Load Tweet and Generate Dataframe

In [ ]:
tweet_objects = proc.loadTweetObjects(DIRECTORY)
encoding = True #Use False on Macs for better results
df = proc.convertTweetsToDataframe(tweet_objects, ENGLISH, encoding)

### Extract Potential Cashtags

In [ ]:
ctdf = proc.extractPossibleCashtags(df)

### Removing Noisy Tweets

In [ ]:
'''
*** Tweets often use popular hashtags with unrelated topics.
*** Noisy words can be identified to use to filter such tweets.
*** Enter these words below in the noisy_terms list.
'''
noisy_terms = []
cldf = proc.removeNoisyTerms(df, noisy_terms)
cldf['original_tweet'][cldf.original_tweet == 'None'] = cldf['tweet']
cldf['tweet'] = cldf['original_tweet']
del cldf['original_tweet']

### Remove Retweets

In [ ]:
cldf_no_RT, cldf = proc.removeRetweets(cldf)

In [ ]:
print(cldf_no_RT.shape[0])

## SPLITTING DATA

### By date

In [ ]:
cldf, SORTED = ana.sortByDate(cldf,SORTED,PRINT_TOP=False,TOP=10)
begin = datetime.datetime(2018,10,2,23,59) #year, month, day, hour, minute
end = datetime.datetime(2019,1,2,23,59)
specific_range_cldf = cldf[cldf.date > begin]
specific_range_cldf = specific_range_cldf[specific_range_cldf.date < end]

### By terms

In [ ]:
terms_of_interest = []
bsdf = proc.findTerms(cldf, terms_of_interest)

## CLUSTERING

In [ ]:
'''
*** Enter additional stop words here.
*** This will ignore the words rather than removing tweets as with noisy Tweets.
'''
ADDITIONAL_STOP_WORDS = ['rt','wa','http','ha','1','ð','3','weâ','â','foâ','amp',\
                        '10','iâ','ieâ','5','2','ï','¾','¾â','itâ']
stop_words = text.ENGLISH_STOP_WORDS.union(org.STOP_WORDS)
stop_words = stop_words.union(ADDITIONAL_STOP_WORDS)

In [ ]:
'''
*** Clustering requires the use of a number of parameters for tuning.
*** These are included below and should be set based on your project.
'''
n_FEATURES = 300
n_TOPICS = 10
n_TOP_WORDS = 10
n_TOP_TWEETS = 10
NGRAM = 3

In [ ]:
cvectors, cvector_feature_names = cluster.countVectorizer(cldf, n_FEATURES, \
                                           NGRAM, stop_words)

In [ ]:
lda, lda_embedding = cluster.LDA(cvectors, n_TOPICS)

In [ ]:
cluster.printLDA(cldf, lda, lda_embedding, cvector_feature_names,\
                   n_TOP_WORDS, n_TOPICS, n_TOP_TWEETS)

In [ ]:
PERPLEXITY = 2.0
cluster.tSNE(lda_embedding, PERPLEXITY, n_TOPICS, title = \
             't-SNE Visualization of Oscar Nominee Tweets LDA')